In [1]:
import pickle5 as pickle
stanza_ents = pickle.load(open('../../data/stanza_ents.pkl', 'rb'))

from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')
model_vocab = list(tokenizer.vocab.keys())

In [2]:
from tqdm.auto import tqdm
ent_in_model_vocab = []
for ent in tqdm(stanza_ents):
    if ent in model_vocab:
        ent_in_model_vocab.append(ent)

for ent in ent_in_model_vocab:
    stanza_ents.remove(ent)

from collections import Counter
s = Counter(stanza_ents)
sorted_ent_counts = dict(sorted(s.items(), key=lambda item: item[1], reverse=True))

  0%|          | 0/2880 [00:00<?, ?it/s]

In [14]:
import wikipedia
from collections import defaultdict

'''
selected_ents = ['MERS-CoV', 'SARS', 'coronavirus', 'MERS',
                'airway', 'COVID-19', 'human adenovirus type 55', 'CHIKV', 'HIV-1', 'carrageenan']
'''
no_of_ents_to_select = 10
no_of_results_per_entity = 1

selected_ents_text_dict = defaultdict(list)

for ent in tqdm(sorted_ent_counts.keys()):
    if len(selected_ents_text_dict) == no_of_ents_to_select:
        break
    
    search_results = wikipedia.search(str(ent), results=no_of_results_per_entity)
    
    for res in search_results:
        try:
            selected_ents_text_dict[ent].append(wikipedia.page(res, auto_suggest=False).content)          
        except:
            continue
    
    if len(selected_ents_text_dict[ent]) != no_of_results_per_entity:
        selected_ents_text_dict.pop(ent)

  0%|          | 0/1124 [00:00<?, ?it/s]

In [16]:
wikipedia.search('carrageenan', results=3)

['Carrageenan', 'Seaweed farming', 'Chondrus crispus']

In [15]:
selected_ents_text_dict.keys()

dict_keys(['MERS-CoV', 'SARS', 'the study', 'coronavirus', 'MERS', 'COVID-19', 'airway', 'human adenovirus type 55', 'HAdV-55', 'carrageenan'])

In [ ]:
import pandas as pd
pd.DataFrame(zip(selected_ents, texts), columns = ['ent', 'text']).to_csv('mini_corpus.csv', index=False)